In [1]:
from langchain_openai.llms import OpenAI

llm = OpenAI()

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário:
    {question}
    """
)

In [3]:
print(prompt_template.format(question="O que é um SaaS?"))


    Responda a seguinte pergunta do usuário:
    O que é um SaaS?
    


In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {question}
    """
)

In [5]:
print(prompt_template.format(question="O que é um SaaS?", n_palavras=15))


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é um SaaS?
    


In [6]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
    {question}
    """, partial_variables={"n_palavras": 15}
)

In [7]:
print(prompt_template.format(question="O que é LangChain?"))


    Responda a seguinte pergunta do usuário em até 15 palavras:
    O que é LangChain?
    


## Utilizando multiplos prompts

In [8]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_palavras} palavras.
"""
)

template_line_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_linhas} linhas.                                                 
"""
)

template_idioma = PromptTemplate.from_template("""
Retorne a resposta em {idioma}.                                                 
"""
)

template_final = (template_word_count + template_line_count + template_idioma +
                  "Responda a pergunta seguindo as instruções {pergunta}")

In [9]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] input_types={} partial_variables={} template='\nResponda a pergunta em até {n_palavras} palavras.\n\nResponda a pergunta em até {n_linhas} linhas.                                                 \n\nRetorne a resposta em {idioma}.                                                 \nResponda a pergunta seguindo as instruções {pergunta}'


In [10]:
prompt_final = template_final.format(n_palavras=15, n_linhas=1, idioma="inglês", pergunta="O que é o Sol?")
llm.invoke(prompt_final)

'\n\nThe Sun is the star at the center of our solar system that provides light and heat to Earth.'

In [11]:
print(prompt_final)


Responda a pergunta em até 15 palavras.

Responda a pergunta em até 1 linhas.                                                 

Retorne a resposta em inglês.                                                 
Responda a pergunta seguindo as instruções O que é o Sol?


### Templates para Chat

In [12]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem é você?")

[HumanMessage(content='Essa é minha dúvida: Quem é você?', additional_kwargs={}, response_metadata={})]

In [13]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente irônico e se chama {nome_assistente}"),
        ("human", "Olá, como vai?"),
        ("ai", "Estou bem, como posso lhe ajudar?"),
        ("human", "{pergunta}")
    ]
)

In [14]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], input_types={}, partial_variables={}, template='Você é um assistente irônico e se chama {nome_assistente}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Olá, como vai?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Estou bem, como posso lhe ajudar?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='{pergunta}'), additional_kwargs={})])

In [15]:
chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?")

[SystemMessage(content='Você é um assistente irônico e se chama BotX', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá, como vai?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Estou bem, como posso lhe ajudar?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual seu nome?', additional_kwargs={}, response_metadata={})]

In [16]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente="BotX", pergunta="Qual seu nome?"))

AIMessage(content='Meu nome é BotX, o assistente irônico. Como posso ser sarcástico hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 55, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3x2yz9yHgYpK0pd558zkoRz3Q1zn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4e5daad6-c0ca-4e65-8101-3583f08f2003-0', usage_metadata={'input_tokens': 55, 'output_tokens': 23, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Few Shot Prompting

In [17]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [18]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

'Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n'

In [19]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [20]:
print(prompt.format(input="Quem é melhor, Messi ou Cristiano Ronaldo?"))

Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 


Pergunta Quem nasceu primeiro, Charles Darwin ou Albert Einstein?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 


Pergunta Quem foi o pai de Napoleão Bonaparte?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte?

In [21]:
llm.invoke(prompt.format(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?"))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Messi fez em sua carreira? \nResposta intermediária: Messi fez 731 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Cristiano Ronaldo fez em sua carreira? \nResposta intermediária: Cristiano Ronaldo fez 778 gols em sua carreira. \nEntão a resposta final é: Cristiano Ronaldo'

### Few Shot Prompting em Chat

In [22]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

In [23]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
     ("ai", "{resposta}")]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Qual é a maior montanha do mundo, o Monte Everest ou o K2?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n', additional_kwargs={}, response_metadata={})]


In [24]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?")
# few_shot_template.format_messages()

In [25]:
chat.invoke(prompt)

AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Messi marcou em sua carreira? \nResposta intermediária: Até o momento, Lionel Messi marcou mais de 700 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Cristiano Ronaldo marcou em sua carreira? \nResposta intermediária: Cristiano Ronaldo também marcou mais de 700 gols em sua carreira. \nEntão a resposta final é: Ambos marcaram a quantidade de gols semelhante em suas carreiras.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 520, 'total_tokens': 650, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3x328lh6OS6p1BobCJLpuUQDn7T0', 'service_tier': 'default',